In [1]:
!pip install yfinance

In [2]:
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import pandas as pd

In [3]:
def getData(ticker):
    """Function to pull data from Yahoo faster"""
    data = yf.Ticker(ticker+"-USD").history(period='max').reset_index()[['Date', 'Open']].rename(columns={'Open': ticker})
    return data

In [4]:
btcdata = getData("BTC")
ethdata = getData("ETH")
xmrdata = getData("XMR")

df = btcdata.merge(ethdata, on='Date')
df = df.merge(xmrdata, on='Date').set_index('Date')
df = df/df.iloc[0] # Normalize data for Sharpe ratio

df

In [5]:
dfPct = df.pct_change() # Daily change in price
meanPct = dfPct.mean() # Mean for Sharpe ratio
covMatrix = df.cov() # How much assets vary together 

display(dfPct)
display(meanPct)
covMatrix

In [6]:
assets = ['btc', 'xmr', 'eth']
num_iterations = 10000
simulation_res = np.zeros((4+len(assets)-1,num_iterations))

In [7]:
for i in range(num_iterations):
    """Calculating standard deviation and returns for random portfolios."""
    weights = np.array(np.random.random(3))
    weights /= np.sum(weights)
    
    portfolioReturn = np.sum(meanPct * weights)
    portfolioStd = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights)))
    
    simulation_res[0, i] = portfolioReturn
    simulation_res[1, i] = portfolioStd
    simulation_res[2, i] = simulation_res[0, i] / simulation_res[1, i]
    
    for j in range(len(weights)):
        simulation_res[j+3, i] = weights[j]
        
simulation_res.T

In [8]:
dfplot = pd.DataFrame(simulation_res.T, columns=['avg_ret', 'std', 'sharpe', assets[0], assets[1], assets[2]])
dfplot

In [9]:
g = sns.scatterplot(data=dfplot, x='std', y='avg_ret', hue='sharpe', size='sharpe')
g.set(ylabel='Average returns', xlabel='Volatility', title='Return per volatility (Sharpe ratio)')

After going through the calculation of the Sharpe ratio and the generation of random portfolios, we see that there is a correlation between risk (measured by the volatility) and the average returns of a given portfolio with Bitcoin, Ethereum and Monero.